# Let's make a random star generator!

## Notes

https://www.reddit.com/r/DnDBehindTheScreen/comments/8efeta/a_random_star_chart_generator/
http://www.stargazing.net/david/constel/howmanystars.html

Altitudes! 

- ~~altitude bins --> altitude window~~
- ~~have a horizon window with panes; how tall is the window = altitude~~
- ~~can we move the window around in the sky? always will be horizontal, but center point = azimuth, change minimum and maximum settings of the altitude~~
- ~~need to make sure it works--currently setting fairly reasonable altitude requirements gives empty RSCs???~~
    - ~~ok I think this is a function of looking due S and having only Sirius-like data; they all transit at the same alt at az of 180...?~~
- have default window that can be overwritten

Random stars!

- make sure to save them!
-  print a dictionary with Dec and RA so we can re-run that choice exactly
    - save the sandom seed? 
    - save a spherical plot of the decs and RAs
        - plot a line showing circumpolar and also visible ever
- right now, generating stars above the horizon; but also generating in wrong places 
    - up to 100 stars within correct dec (check altitude window tmax magnitude = o convert to dec max, min)
    - make sure to note min max dec in the output as well!
    - generate a list file for Stellarium to be able to plot "novae"
        - maybe Sarah has a file to show me so I can format it correctly? JSON file in email!
        - HAVE to leave actual novae list at the end (can't remember why)
        - max mag = min mag --> so the nova doesn't decay
        - other ways to do this with stellarium?


Making choices!

- brightest vs dimmest? 
- brightest vs two other considerations: 
    - run with all possibilities, then make three different choices:
        - brightest star wins
        - hr aqa ib wins
        - the most accurately placed star wins (most bang on with position of any bin; maybe compare narrow and wide gaps between bins)
        - choose the more common star; appears in most other tables
            - this may need a two step: run once, make a sheet with all posibilities, create a dictionary with how many time each star appears
            - then, create three more sheets with the above runs, including the last (which uses the dictionary to make a choice) 
- code right now doesn't make a difference between stars on a line and between two lines!



## Code

In [39]:
#from random import seed
#from random import gauss
import random
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
import pandas as pd

In [45]:
# apparently it's way faster to make a list of coordiantes and then add one to skycoord obejct https://docs.astropy.org/en/stable/coordinates/skycoord.html  
# let's try to implement this differently 

def randomStar():
    '''
    Make a single random star in Dec and RA.
    '''
    #minimum dec visible 
    lat = 25.6989 # lat of Luxor in degrees
    maxnum = 0.5 * (np.cos(np.pi * lat / 180) + 1) # for generating stars visible at Luxor below the hemisphere
    #select a random point uniformly across a hemisphere +
    x = random.random()
    y = random.uniform(0, maxnum) #0-0.5: above horizon, 0.5-1 below horizon
    phi = np.arccos(2*y-1) - np.pi/2
    theta = 2 * np.pi * x
    #mag = random.uniform(1,6) # magnitude of star (if we want it)
    # make them into RA and Dec
    RA = Angle(theta * u.radian).hour
    Dec = Angle(phi * u.radian).deg
    # obj = SkyCoord(ra=RA, dec=Dec, unit="deg")
    return (RA, Dec)

def randomStarField(num):
    '''
    Make a number = num of random stars in Dec and RA to analyze with decanOpy.
    It is expected that num < 100, otherwise the naming convention will be wrong. 
    To add more, change "{:02.0f}" to "{:0X.0f}", where X is the number of digits. 
    '''
    # initalize empty key lists
    star_names = []
    obj_list = []
    RA_list = []
    Dec_list = []
    mag_list = []
    hd_list = ["Julian Date", "Local Date and Time", "Sun Azimuth", "Sun Altitude"]
    for i in range(0, num):
        name = "R" + "{:04.0f}".format(i)
        star_names.append(name)
        (RA, Dec) = randomStar()
        RA_list.append(RA)
        Dec_list.append(Dec)
        mag_list.append(round(random.uniform(0, 6), 2))
        #obj = randomStar()
        #obj_list.append(obj)
        hd_list.append(name + " Azimuth")
        hd_list.append(name + " Altitude")
    # save    
    df = pd.DataFrame({"Name" : star_names, 
                    "RA" : RA_list,
                    "Dec": Dec_list,
                    "Mag": mag_list})    
    df.to_csv('out.csv', index=False)
    obj_list = SkyCoord(RA_list * u.hour, Dec_list * u.deg)
    return(obj_list, hd_list)    

In [46]:
(obj_list, hd_list) = randomStarField(10) 

In [36]:
for obj in obj_list:
    print(obj)

<SkyCoord (ICRS): (ra, dec) in deg
    (219.93244878, -59.96474217)>
<SkyCoord (ICRS): (ra, dec) in deg
    (338.64422247, 17.1978938)>
<SkyCoord (ICRS): (ra, dec) in deg
    (302.25424406, -18.01063052)>
<SkyCoord (ICRS): (ra, dec) in deg
    (82.9208859, -3.9636864)>
<SkyCoord (ICRS): (ra, dec) in deg
    (300.50849256, -56.56860577)>
<SkyCoord (ICRS): (ra, dec) in deg
    (269.13805246, 4.4376372)>
<SkyCoord (ICRS): (ra, dec) in deg
    (254.0097394, -4.63950556)>
<SkyCoord (ICRS): (ra, dec) in deg
    (306.31387829, 15.13278387)>
<SkyCoord (ICRS): (ra, dec) in deg
    (218.12260488, -1.93501628)>
<SkyCoord (ICRS): (ra, dec) in deg
    (355.15599395, -36.31806203)>


In [37]:
hd_list

['Julian Date',
 'Local Date and Time',
 'Sun Azimuth',
 'Sun Altitude',
 'R0000 Azimuth',
 'R0000 Altitude',
 'R0001 Azimuth',
 'R0001 Altitude',
 'R0002 Azimuth',
 'R0002 Altitude',
 'R0003 Azimuth',
 'R0003 Altitude',
 'R0004 Azimuth',
 'R0004 Altitude',
 'R0005 Azimuth',
 'R0005 Altitude',
 'R0006 Azimuth',
 'R0006 Altitude',
 'R0007 Azimuth',
 'R0007 Altitude',
 'R0008 Azimuth',
 'R0008 Altitude',
 'R0009 Azimuth',
 'R0009 Altitude']

In [ ]:
# how many stars
df = pd.DataFrame(data=np.empty((13,7), dtype=str))

### Old version (slower)

In [26]:
# apparently it's way faster to make a list of coordiantes and then add one to skycoord obejct https://docs.astropy.org/en/stable/coordinates/skycoord.html  

def randomStar():
    '''
    Make a single random star in Dec and RA.
    '''
    #minimum dec visible 
    lat = 25.6989 # lat of Luxor in degrees
    maxnum = 0.5 * (np.cos(np.pi * lat / 180) + 1) # for generating stars visible at Luxor below the hemisphere
    #select a random point uniformly across a hemisphere +
    x = random.random()
    y = random.uniform(0, maxnum) #0-0.5: above horizon, 0.5-1 below horizon
    phi = np.arccos(2*y-1) - np.pi/2
    theta = 2 * np.pi * x
    #mag = random.uniform(1,6) # magnitude of star (if we want it)
    # make them into RA and Dec
    RA = Angle(theta * u.radian).hour
    Dec = Angle(phi * u.radian).deg
    obj = SkyCoord(ra=RA, dec=Dec, unit="deg")
    return obj

def randomStarField(num):
    '''
    Make a number = num of random stars in Dec and RA to analyze with decanOpy.
    It is expected that num < 100, otherwise the naming convention will be wrong. 
    To add more, change "{:02.0f}" to "{:0X.0f}", where X is the number of digits. 
    '''
    # initalize empty key lists
    star_names = []
    obj_list = []
    hd_list = ["Julian Date", "Local Date and Time", "Sun Azimuth", "Sun Altitude"]
    for i in range(0, num):
        name = "R" + "{:04.0f}".format(i)
        star_names.append(name)
        obj = randomStar()
        obj_list.append(obj)
        hd_list.append(name + " Azimuth")
        hd_list.append(name + " Altitude")
    return(obj_list, hd_list)    

In [79]:
"{:04.0f}".format(1)

'0001'

In [27]:
timeit (obj_list, hd_list) = randomStarField(300)   

167 ms ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [87]:
hd_list

['Julian Date',
 'Local Date and Time',
 'Sun Azimuth',
 'Sun Altitude',
 'R0000 Azimuth',
 'R0000 Altitude',
 'R0001 Azimuth',
 'R0001 Altitude',
 'R0002 Azimuth',
 'R0002 Altitude',
 'R0003 Azimuth',
 'R0003 Altitude',
 'R0004 Azimuth',
 'R0004 Altitude',
 'R0005 Azimuth',
 'R0005 Altitude',
 'R0006 Azimuth',
 'R0006 Altitude',
 'R0007 Azimuth',
 'R0007 Altitude',
 'R0008 Azimuth',
 'R0008 Altitude',
 'R0009 Azimuth',
 'R0009 Altitude',
 'R0010 Azimuth',
 'R0010 Altitude',
 'R0011 Azimuth',
 'R0011 Altitude',
 'R0012 Azimuth',
 'R0012 Altitude',
 'R0013 Azimuth',
 'R0013 Altitude',
 'R0014 Azimuth',
 'R0014 Altitude',
 'R0015 Azimuth',
 'R0015 Altitude',
 'R0016 Azimuth',
 'R0016 Altitude',
 'R0017 Azimuth',
 'R0017 Altitude',
 'R0018 Azimuth',
 'R0018 Altitude',
 'R0019 Azimuth',
 'R0019 Altitude',
 'R0020 Azimuth',
 'R0020 Altitude',
 'R0021 Azimuth',
 'R0021 Altitude',
 'R0022 Azimuth',
 'R0022 Altitude',
 'R0023 Azimuth',
 'R0023 Altitude',
 'R0024 Azimuth',
 'R0024 Altitude',
 '

In [21]:
obj_list

[<SkyCoord (ICRS): (ra, dec) in deg
     (19.39467112, -23.48113324)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (21.10701595, 13.29503413)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (23.79443314, 14.11380342)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (5.10823274, -39.30501788)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (20.23068543, 57.09844331)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (7.23627972, -4.50609326)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (10.65918864, -16.2073017)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (22.57041514, -36.48761827)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (2.39923739, -31.59871229)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (23.02003454, -33.44610844)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (20.60449898, 41.14727005)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (13.77622614, -32.57585794)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (9.46885441, 14.13579394)>,
 <SkyCoord (ICRS): (ra, dec) in deg
     (5.66350621, 51.16255503)>,
 <SkyCoord (ICRS): 

In [ ]:
# ok, still need to integrate into mockrun.py 